##1 Poker Set_up

In [ ]:
import matplotlib.pyplot as plt
import random
import numpy as np
from itertools import combinations
import pandas as pd

from treys import Card
from treys import Evaluator
evaluator = Evaluator()


##################################################################################
################################ Card Rank ######################################
##################################################################################

suits = ['S','H','D','C']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
deck = [(rank, suit) for rank in ranks for suit in suits]

# Define the rank of each hand
hand_rank = [
    ("Royal flush", lambda hand: is_royal_flush(hand)),
    ("Straight flush", lambda hand: is_straight_flush(hand)),
    ("Four of a kind", lambda hand: is_four_of_a_kind(hand)),
    ("Full house", lambda hand: is_full_house(hand)),
    ("Flush", lambda hand: is_flush(hand)),
    ("Straight", lambda hand: is_straight(hand)),
    ("Three of a kind", lambda hand: is_three_of_a_kind(hand)),
    ("Two pair", lambda hand: is_two_pair(hand)),
    ("Pair", lambda hand: is_pair(hand)),
    ("High card", lambda hand: True)
]

# Define the functions to check for each hand rank
def is_royal_flush(hand):
    return is_flush(hand) and set([rank for rank, suit in hand]) == set(["T", "J", "Q", "K", "A"])

def is_straight_flush(hand):
    return is_flush(hand) and is_straight(hand)

def is_four_of_a_kind(hand):
    ranks = [rank for rank, suit in hand]
    for rank in ranks:
        if ranks.count(rank) == 4:
            return True
    return False

def is_full_house(hand):
    return is_three_of_a_kind(hand) and is_pair(hand)


def is_flush(hand):
    suits = [suit for rank, suit in hand]
    if len(set(suits)) == 1:
        return True
    return False


def is_straight(hand):
    ranks = [rank for rank, suit in hand]
    replace_mapping = {'K': '13',
                       'Q': '12',
                       'J': '11',
                       'T': '10',
                       'A': '1'}
    ranks = [replace_mapping.get(rank, rank) for rank in ranks ]
    ranks_int = [eval(i) for i in ranks]
    max_ranks =max(ranks_int)
    min_ranks = min(ranks_int)
    if (max_ranks - min_ranks == 4) and len(set(ranks)) == 5:
        return True
    return False

def is_three_of_a_kind(hand):
    ranks = [rank for rank, suit in hand]
    for rank in ranks:
        if ranks.count(rank) == 3:
            return True
    return False

def is_two_pair(hand):
    ranks = [rank for rank, suit in hand]
    pairs = set([rank for rank in ranks if ranks.count(rank) == 2])
    if len(pairs) == 2:
        return True
    return False

def is_pair(hand):
    ranks = [rank for rank, suit in hand]
    for rank in ranks:
        if ranks.count(rank) == 2:
            return True
    return False

# Define a function to rank a hand
def rank_hand(hand):
    for rank_name, rank_function in hand_rank:
        if rank_function(hand):
            return rank_name
##################################################################################
################################ Hand Strength ###################################
##################################################################################
def cards_dealt():
    suits = ['S','H','D','C']
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
    deck = [(rank, suit) for rank in ranks for suit in suits]

    # Shuffle the deck
    random.shuffle(deck)

    # Deal two cards to each player
    player1_hand = [deck.pop(), deck.pop()]
    player2_hand = [deck.pop(), deck.pop()]

    # Deal five community cards
    community_cards = [deck.pop() for _ in range(5)]

    player1_full_hand = player1_hand + community_cards
    player2_full_hand = player2_hand + community_cards

    return player1_full_hand,player2_full_hand

# Use 0-1 to represent the Hand Strength

map_dict ={
'Royal flush':1,
'Straight flush':0.9,
'Four of a kind':0.8,
'Full house':0.7,
'Flush':0.6,
'Straight': 0.5,
'Three of a kind':0.4,
'Two pair':0.3,
'Pair'  : 0.2,
'High card':0.1,
}

def hand_strength (full_hand):
    HS = 0
    combination = combinations(full_hand,5)
    for combo in combination:
        combo_rank = rank_hand(combo)
        combo_rank_num =map_dict[combo_rank]
        if combo_rank_num > HS:
            HS = combo_rank_num
    return HS

def deusces_value (cards):

    suits_map = {'H':'h',
                'D':"d",
                'S':"s",
                 'C':'c',}
    ranks = [rank for rank, suit in cards]
    suits = [suit for rank, suit in cards]
    deusces_suits = [suits_map.get(suit, suit) for suit in suits]
    deusces_hand = [r + s for r,s in zip(ranks,deusces_suits)]
    board = deusces_hand[2:]
    hand = deusces_hand[:2]
    d_board = [Card.new(i) for i in board]
    d_hand =  [Card.new(i) for i in hand]
    deuces_value = (evaluator.evaluate(d_board, d_hand))
    return deuces_value

## 2 Simulation Play

In [ ]:
#################################################################################
#####################################Game Settings###################################
##################################################################################
def hand_level(hs):
  if hs <0.4:
    return 'Weak'
  if hs >= 0.8:
    return 'Top'
  if hs ==0.4 or hs ==0.5:
    return "Medium"
  else: # 0.6,0.7
    return "Strong"


def get_strategy_B_simulation(HS):
    hl = hand_level(HS)
    i = random.random()

    if hl == 'Weak':
      if i <= 0.7:
          return 'C'
      else:
          return 'B'
    
    elif hl == 'Top' :
      return 'B'
      #if i <= 0.1:
          #return 'C'
      #else:
          #return 'B'
    
    elif hl == 'Strong':
      #return 'B'

      if i <= 0.1:
          return 'C'
      else:
          return 'B'

    else: #hl == 'Medium
      if i <= 0.3:
          return 'C'
      else:
          return 'B'
    
A_strategy_map ={
  'Medium ': [0.98, 0.02],
 'Medium B': [0.01,0.99],
 'Medium C': [0.01, 0.99],
 'Medium CB':[0.01,0.99],

 'Strong ': [0.97, 0.03],
'Strong B': [0.01,0.99],
 'Strong C': [0.01,0.99],
 'Strong CB': [0,1],

 'Top ': [0.6, 0.4],
 'Top B': [0.01,0.99],
 'Top C': [0,1],
 'Top CB':[0.01,0.99],

 'Weak ': [0.01,0.99],
 'Weak B': [0.01,0.99],
 'Weak C': [0.01,0.99],
 'Weak CB': [0.01,0.99]}



def A_strategy(HS,history):
    hs = hand_level(HS)
    strategy_key = str(hs) + " " + history
    strategy = A_strategy_map[strategy_key]
    bet_prob = strategy[1]
    i = random.random()
    if i <= bet_prob:
        return "B"
    else:
        return "C"

W = 50
## first mover is A and win
payoff_tracker_player_A_map ={
'BB': 2/W,
'BC': 1/W,
'CC': 1/W,
'CBB':2/W,
'CBC': -1/W
}

def payoff_tracker_player_A(first_mover,history,full_hand_A, full_hand_B,starting_stack_A):
    hs_A = deusces_value (full_hand_A)
    hs_B = deusces_value (full_hand_B)
    if history in ['BB','CC','CBB']:
        if hs_A < hs_B:
            return payoff_tracker_player_A_map[history] * starting_stack_A
        else:
            return - payoff_tracker_player_A_map[history] * starting_stack_A
    else: 
        if first_mover == 'A':
            return payoff_tracker_player_A_map[history] * starting_stack_A
        else:
            return - payoff_tracker_player_A_map[history] * starting_stack_A

    
def terminal_check(history):

    if history in ['BB','CC','CBB','BC',"CBC"]:
        return True
    else:
        False


win_rate ={
"BB_Win":0,
"BB_Lose":0,
"CC_Win":0,
"CC_Lose":0,
"CBB_Win":0,
"CBB_Lose":0,
'BC_Win':0,
'BC_Lose':0,
"CBC_Win":0,
"CBC_Lose":0,
}

terminal_dict ={
'BB': 0,
'CC':0,
'CBB':0,
'BC':0,
"CBC":0
}

paypff_dict={
'BB': 0,
'CC':0,
'CBB':0,
'BC':0,
"CBC":0
}


util_a_sum = 0
N = 10000
num_iterations = N
player_list =['A','B']
# chips
stack_count = 1000
i = 0
j = 0

for _ in range(num_iterations):
        
        if stack_count >=2000:
            i +=1
            stack_count = 1000
        
        if stack_count <=0:
            j +=1
            stack_count = 1000
        
    
        # A and B take turns to action first
        first_action_player = player_list[_ % 2]
        full_hand_1, full_hand_2 = cards_dealt()
        HS_A = hand_strength (full_hand_1)
        HS_B = hand_strength (full_hand_2)
        history =''

        if first_action_player == 'A':
            A_move_1 = A_strategy(HS_A,history)
            history +=A_move_1
            B_move_2 = get_strategy_B_simulation(HS_B)
            history +=B_move_2
       
            if terminal_check(history):
                a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)

                util_a_sum +=a_payoff
                paypff_dict[history]+=a_payoff
                terminal_dict[history]+=1
                stack_count +=a_payoff


                
                if a_payoff > 0:
                    win_rate[history+ "_Win"] +=1
                else:
                    win_rate[history+ "_Lose"] +=1
            else:
                A_move_3 = A_strategy(HS_A,history)
                history +=A_move_3
                if terminal_check(history):
                    a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)

                    util_a_sum +=a_payoff
                    paypff_dict[history]+=a_payoff
                    terminal_dict[history]+=1
                    stack_count +=a_payoff
            

                    if a_payoff > 0:
                        win_rate[history+ "_Win"] +=1
                    else:
                        win_rate[history+ "_Lose"] +=1


        else :
       
            B_move_1 = get_strategy_B_simulation(HS_B)
            history +=B_move_1
            A_move_2 = A_strategy(HS_A,history)
            history +=A_move_2
            if terminal_check(history):
                a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)
              
                util_a_sum +=a_payoff 
                paypff_dict[history]+=a_payoff
                terminal_dict[history]+=1
                stack_count +=a_payoff
             


                if a_payoff > 0:
                    win_rate[history+ "_Win"] +=1
                else:
                    win_rate[history+ "_Lose"] +=1

            else:
                B_move_3 = get_strategy_B_simulation(HS_B)
                history +=B_move_3
                if terminal_check(history):
                    a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)                 
                    util_a_sum +=a_payoff 
                    paypff_dict[history]+=a_payoff
                    terminal_dict[history]+=1
                    stack_count +=a_payoff
                 
                    
                    if a_payoff > 0:
                        win_rate[history+ "_Win"] +=1
                    else:
                        win_rate[history+ "_Lose"] +=1

print(i)
print(j)

## 3 Results Format

In [ ]:
df_terminal_count = pd.DataFrame(terminal_dict.items(), columns=['History', 'Reach_count'])
df_terminal_count['Reach_prob']= df_terminal_count['Reach_count'] / N
reach_count = df_terminal_count['Reach_count'].tolist()
reach_prob = df_terminal_count['Reach_prob'].tolist()
new_reach_count = [element for element in reach_count  for i in range(2)]
new_reach_prob = [element for element in reach_prob   for i in range(2)]
#####################################################################################
#####################################################################################
df_result = pd.DataFrame(win_rate.items(), columns=['History', 'Win_Lose'])
df_result['T_reach_count'] = new_reach_count
df_result['T_reach_prob'] = new_reach_prob
df_result['T_win_rate'] = df_result['Win_Lose'] / df_result['T_reach_count']
df_result['T_win_rate'] = df_result['T_win_rate'].fillna(0)
df_result['Probability'] = df_result['T_reach_prob'] * df_result['T_win_rate']
#####################################################################################
#####################################################################################
#payoff_rate = [202/200,198/200,201/200,199/200,202/200,198/200,201/200,199/200,201/200,199/200]
payoff_rate = [52/50,48/50,51/50,49/50,52/50,48/50,51/50,49/50,51/50,49/50]
df_result['Payoff_rate'] =payoff_rate

df_result['exp_action'] = df_result['Payoff_rate'] * df_result['Probability']
df_result['ta_action'] = df_result['Payoff_rate'] ** df_result['Probability']

In [ ]:
check = sum(df_result['Probability'].tolist())
print(check)
print('---------------------')
#per_round = util_a_sum / i
#print(((per_round+50)/50)**(N/i))
print(util_a_sum)
print(((util_a_sum+1000)/1000)**(1/N))
exp_action = sum(df_result['exp_action'].tolist())
print(exp_action)
print('---------------------')
df_result['ta_action_cumprob']  = df_result['ta_action'].cumprod(axis = 0)
df_result['exp_action_cumsum']  = df_result['exp_action'].cumsum(axis = 0)
df_result

## 4 Plot

In [ ]:
iterations = []
chip_count_1 = []
chip_count_2 = []
chip_count_3 = []
chip_count_4 = []
chip_count_5 = []
chip_count_6 = []
chip_count_7 = []
chip_count_8 = []
chip_count_9 = []
chip_count_10 = []

for _ in range(num_iterations):
        iterations.append(_)
        #chip_count_1.append(stack_count)
        #chip_count_2.append(stack_count)
        #chip_count_3.append(stack_count)
        #chip_count_4.append(stack_count)
        #chip_count_5.append(stack_count)
        #chip_count_6.append(stack_count)
        chip_count_7.append(stack_count)
        #chip_count_8.append(stack_count)
        #chip_count_9.append(stack_count)
        #chip_count_10.append(stack_count)

        #if stack_count >=2000:
            #i +=1
            #stack_count = 1000
        
        #if stack_count <=0:
            #j +=1
            #stack_count = 1000
        
    
        # A and B take turns to action first
        first_action_player = player_list[_ % 2]
        full_hand_1, full_hand_2 = cards_dealt()
        HS_A = hand_strength (full_hand_1)
        HS_B = hand_strength (full_hand_2)
        history =''

        if first_action_player == 'A':
  
            A_move_1 = A_strategy(HS_A,history)
            history +=A_move_1
            B_move_2 = get_strategy_B_simulation(HS_B)
            history +=B_move_2
       
            if terminal_check(history):
                a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)
             
                util_a_sum +=a_payoff
                paypff_dict[history]+=a_payoff
                terminal_dict[history]+=1
                stack_count +=a_payoff
          

                if a_payoff > 0:
                    win_rate[history+ "_Win"] +=1
                else:
                    win_rate[history+ "_Lose"] +=1
            else:
                A_move_3 = A_strategy(HS_A,history)
                history +=A_move_3
                if terminal_check(history):
                    a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)
              
                    util_a_sum +=a_payoff
                    paypff_dict[history]+=a_payoff
                    terminal_dict[history]+=1
                    stack_count +=a_payoff
                  

                    if a_payoff > 0:
                        win_rate[history+ "_Win"] +=1
                    else:
                        win_rate[history+ "_Lose"] +=1


        else :
           
            B_move_1 = get_strategy_B_simulation(HS_B)
            history +=B_move_1
            A_move_2 = A_strategy(HS_A,history)
            history +=A_move_2
            if terminal_check(history):
                a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)
              
                util_a_sum +=a_payoff 
                paypff_dict[history]+=a_payoff
                terminal_dict[history]+=1
                stack_count +=a_payoff
     


                if a_payoff > 0:
                    win_rate[history+ "_Win"] +=1
                else:
                    win_rate[history+ "_Lose"] +=1

            else:
                B_move_3 = get_strategy_B_simulation(HS_B)
                history +=B_move_3
                if terminal_check(history):
                    a_payoff = payoff_tracker_player_A(first_action_player,history,full_hand_1, full_hand_2,stack_count)
                  
                    util_a_sum +=a_payoff 
                    paypff_dict[history]+=a_payoff
                    terminal_dict[history]+=1
                    stack_count +=a_payoff
                 
                    
                    if a_payoff > 0:
                        win_rate[history+ "_Win"] +=1
                    else:
                        win_rate[history+ "_Lose"] +=1

In [ ]:
df_plot = pd.DataFrame(columns = ['#Hand','Expected_Chips','Simulation_1',
                                  'Simulation_2','Simulation_3','Simulation_4',
                                  'Simulation_5','Simulation_6','Simulation_7',
                                  'Simulation_8','Simulation_9','Simulation_10'])


In [ ]:
Simulation_k = chip_count_k

In [ ]:
df_plot['#Hand'] = iterations
Expected_Chips =[]
for i in range(0,5000):
  term = 1000 *1.0076 ** i
  Expected_Chips.append(term)

df_plot['Expected_Chips'] = Expected_Chips
df_plot['Simulation_1'] = Simulation_1
df_plot['Simulation_2'] = Simulation_2
df_plot['Simulation_3'] = Simulation_3
df_plot['Simulation_4'] = Simulation_4
df_plot['Simulation_5'] = Simulation_5
df_plot['Simulation_6'] = Simulation_6
df_plot['Simulation_7'] = Simulation_7
df_plot['Simulation_8'] = Simulation_8
df_plot['Simulation_9'] = Simulation_9
df_plot['Simulation_10'] =Simulation_10

In [ ]:
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Expected_Chips'], label = "Expectation")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_1'], label = "Sim_1")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_2'], label = "Sim_2")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_3'], label = "Sim_3")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_4'], label = "Sim_4")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_5'], label = "Sim_5")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_6'], label = "Sim_6")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_7'], label = "Sim_7")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_8'], label = "Sim_8")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_9'], label = "Sim_9")
plt.plot(df_plot_copy['#Hand'], df_plot_copy['Simulation_10'], label = "Sim_10")
plt.legend()
plt.show()